In [1]:
from tmc_summarizer import TMC_File 
tmc = TMC_File('/Volumes/GoogleDrive/My Drive/tmc_copy/1_Central_Ave.xls')

Reading 1_Central_Ave.xls


/Users/markmorley/repos/tmc-summarizer/tmc_summarizer/data_model.py:292: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = None
/Users/markmorley/repos/tmc-summarizer/tmc_summarizer/data_model.py:292: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = None
/Users/markmorley/repos/tmc-summarizer/tmc_summarizer/data_model.py:292: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [2]:
# tmc.peak_hour_factor('am')
tmc.meta

{'location_id': '1',
 'location_name': '162251 - CR 620 Arena Dr & Central Ave',
 'date': '2022-03-24',
 'time': '06:00:00 to 19:00:00',
 'am_peak': '07:15 to 08:15',
 'pm_peak': '15:30 to 16:30',
 'am_peak_raw': (Timestamp('2022-03-24 07:15:00'),
  Timestamp('2022-03-24 08:15:00')),
 'pm_peak_raw': (Timestamp('2022-03-24 15:30:00'),
  Timestamp('2022-03-24 16:30:00')),
 'am_peak_hour_factor': 0.9068493150684932,
 'pm_peak_hour_factor': 0.9402985074626866,
 'leg_nb': 'Central Ave',
 'leg_eb': 'CR 620 Arena Dr',
 'leg_wb': 'CR 620 Arena Dr',
 'filepath': '/Volumes/GoogleDrive/My Drive/tmc_copy/1_Central_Ave.xls'}